In [1]:
import pandas as pd
import datetime
import numpy as np
from catboost import CatBoostRegressor
from surprise import SVD, SVDpp, NMF, CoClustering, SlopeOne, NormalPredictor
from surprise import Reader, Dataset
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy import stats
import dateutil.relativedelta
import matplotlib.pyplot as plt
import seaborn as sns
from cornac.models import SKMeans, MostPop
from cornac.metrics import MAE, RMSE, Precision, Recall, NDCG, AUC, MAP
import cornac
from sklearn.preprocessing import MinMaxScaler

C:\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


FM model is only supported on Linux.
Windows executable can be found at http://www.libfm.org.


In [3]:
cities=['Los', 'NY','Rome','Stockholm','saint']
c_num=4
df = pd.read_csv('C:\Anaconda3\Scripts\Диплом\ONLINE EXCPERIMENT\GIT/FINAL_REVWS_ALL.csv', index_col=0)
df['date'] = pd.to_datetime(df['date'])
#df.shape

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Подготовка данных

In [4]:
def add_new_columns_with_context(df):
    df = df.sort_values('date', ascending=False)
    df.drop_duplicates(subset=['user_name', 'unique_rest'], inplace=True)
    df.fillna(0, inplace=True)
    for col in df.columns:
        if len(df[col].unique()) == 1:
            df.drop(col,inplace=True,axis=1)
    df.drop_duplicates(subset=['user_name', 'unique_rest'], inplace=True)
#оставляем пользователей с количеством отзывов >1
    users = df.groupby('user_name')['rating'].count()[df.groupby('user_name')['rating'].count()>1].index.tolist()
    df = df[df.user_name.isin(users)]
    q_dict = df.drop_duplicates('unique_rest').groupby('city')['dist'].describe()[['25%', '50%', '75%']]
    q_df = q_dict.to_dict('index')
    #по перцентилям 
    df['dist'] = dist2category(df.city, df.dist, q_df)
    #название городов в числа
    city2cat_dict = {x:i for i,x in enumerate(df['city'].unique().tolist())}
    print(city2cat_dict)
    df['city'] = df['city'].apply(lambda x: city2cat_dict[x])
    # текущее количество отзывов у пользователя
    df['cur_count'] = df.sort_values('date').groupby('user_name').cumcount()
    df['cur_count'] = df.cur_count.apply(lambda x: int(cur_count2cur_count_category(x)))
    df['covid'] = df.date.apply(lambda x: func(x))
    return df

In [5]:
def for_city(df):
    #контекст отзыва
    csum_pos = df.sort_values('date').groupby('user_name')['pos'].cumsum()
    ccount_pos = df.sort_values('date').groupby('user_name')['pos'].cumcount()
    cmean_pos = (csum_pos/ccount_pos).replace([np.inf, -np.inf], np.nan).fillna(0)
    csum_neu = df.sort_values('date').groupby('user_name')['neu'].cumsum()
    ccount_neu = df.sort_values('date').groupby('user_name')['neu'].cumcount()
    cmean_neu = (csum_neu/ccount_neu).replace([np.inf, -np.inf], np.nan).fillna(0)
    csum_neg = df.sort_values('date').groupby('user_name')['neg'].cumsum()
    ccount_neg = df.sort_values('date').groupby('user_name')['neg'].cumcount()
    cmean_neg = (csum_neg/ccount_neg).replace([np.inf, -np.inf], np.nan).fillna(0)
    df['cur_cmean_pos'] = cmean_pos
    df['cur_cmean_neu'] = cmean_neu
    df['cur_cmean_neg'] = cmean_neg
    #перцентиль по контексту
    avgs = ['cur_cmean_pos','cur_cmean_neu', 'cur_cmean_neg']
    tdict = df[(df[avgs]!=0).all(axis=1)][avgs].describe().T[['25%', '50%', '75%']].to_dict('index')    
    for col in avgs:
        print(col)
        temp = [cur_sent2cur_sent_category(x, col, tdict) for x in df[col].tolist()]
        df[col] = temp
    df.drop(['text', 'rest_name', 'neg', 'neu', 'pos', 'review_language', 'average_rating',
         'num_all_rev',"country"], axis=1, inplace=True)
    norm = ['user_name','unique_rest','date']
    for col in df.columns:
        if col not in norm:
            df[col] = df[col].astype(int)
    return df

In [6]:
def fi_count(df,bounds):
    fi = dict()
    rmse_CB = []
    for i,k in enumerate(bounds.keys()):
        print(bounds[k][0], bounds[k][1])
        df_train,df_test=create_train_test(df,bounds[k][0], bounds[k][1])
        temp = df[df.date<bounds[k][0]].groupby('unique_rest')['rating'].mean().reset_index().rename(columns={'rating':'average_rating'})
        temp['average_rating'] = temp.average_rating.apply(lambda x: ratata(x))
        temp_avg_rating2cat_dict = {x:i for i,x in enumerate(temp['average_rating'].unique().tolist())}
        temp['average_rating'] = temp['average_rating'].apply(lambda x: temp_avg_rating2cat_dict[x])
        temp1 = df[df.date<bounds[k][0]].groupby('unique_rest')['rating'].count().reset_index().rename(columns={'rating':'num_all_rev'})
        temp1['num_all_rev'] = temp1.num_all_rev.apply(lambda x: func_desc(x, temp1['num_all_rev'].describe()))
        temp1_avg_rating2cat_dict = {x:i for i,x in enumerate(temp1['num_all_rev'].unique().tolist())}
        temp1['num_all_rev'] = temp1['num_all_rev'].apply(lambda x: temp1_avg_rating2cat_dict[x])
        df_train = df_train.merge(temp, on='unique_rest')
        df_test = df_test.merge(temp, on='unique_rest')
        df_train = df_train.merge(temp1, on='unique_rest')
        df_test = df_test.merge(temp1, on='unique_rest')
        X_train = df_train.drop(['rating',
#                              'user_name',
#                              'unique_rest',
                             'date'
                            ], axis=1)
        y_train = df_train['rating']
        X_test = df_test.drop(['rating',
#                            'user_name',
#                            'unique_rest',
                           'date'
                            ], axis=1)
        y_test = df_test['rating']
        cf = list(range(len(X_train.columns.tolist())))
        CB = CatBoostRegressor(iterations=100)
        CB.fit(X_train, y_train, cat_features=cf, verbose=False)
        fi[i] = dict(zip(CB.feature_names_, CB.feature_importances_))
        y_pred = CB.predict(X_test)
        rmse = mean_squared_error(y_pred=y_pred, y_true=df_test.rating, squared=False)
        rmse_CB.append(rmse)
    fi = pd.DataFrame(fi)
    fi_index = fi.sort_values(0, ascending=False)[0].head(20).index.tolist() + fi.sort_values(1, ascending=False)[1].head(20).index.tolist() + fi.sort_values(2, ascending=False)[2].head(20).index.tolist()
    fi_index = list(set(fi_index))
    fi[fi.index.isin(fi_index)].sort_values(1, ascending=False).to_csv(cntry+'fi_v1.csv')
    fi[fi.index.isin(fi_index)].sort_values(1, ascending=False)
    return fi,fi_index

In [4]:
#cntry = 'Russia'
#df = df[df.country==cntry]

In [7]:
df_all=df.copy()

In [20]:
cities=['Los', 'NY','Rome','Stockholm','saint']
df_all=df_all[df_all.city.isin(cities)]
df_all=add_new_columns_with_context(df_all)

{'Rome': 0, 'NY': 1, 'Stockholm': 2, 'Los': 3, 'saint': 4}


In [258]:
#all_city_pred=pd.DataFrame()
for city in range(0,len(cities)):
    print(city)
    df=df_all[df_all.city==city]
    df=df.reset_index(drop=True)
    df=for_city(df)
    fi,fi_index=fi_count(df,bounds)
    cats_rest = [x for x in fi_index if x not in ['user_name', 'unique_rest', 'cur_count', 
                                              'cur_cmean_pos', 'cur_cmean_neu', 'cur_cmean_neg']]
    cats_user = [x for x in fi_index if x in ['cur_count', 'cur_cmean_pos', 'cur_cmean_neu', 'cur_cmean_neg', 'city', 'covid']]
    df['user_context'] = df[cats_user].values.tolist()
    ncats_user = [df[x].nunique() for x in cats_user]
    svd_user_pred=svd_user(df,bounds,city)
    svd_context_pred=svd_context(df,bounds,cats_rest,city)
    svd_cс_pred_2=cc(df,bounds,cats_rest,city)
    all_pred=usual_methods(df,bounds,cats_rest,city)
    all_city_pred=pd.concat([all_city_pred,svd_user_pred,svd_context_pred,svd_cс_pred_2,all_pred], ignore_index=True)
    all_city_pred.to_csv("all_city_pred.csv",sep=";")

3
cur_cmean_pos
cur_cmean_neu
cur_cmean_neg
2020-01-01 00:00:00 2020-03-20 00:00:00
2020-03-20 00:00:00 2020-08-01 00:00:00
2020-08-02 00:00:00 2020-10-01 00:00:00
2020-01-01 00:00:00 2020-03-20 00:00:00
train start
train_end
ranking metrics
2020-03-20 00:00:00 2020-08-01 00:00:00
train start
train_end
ranking metrics
2020-08-02 00:00:00 2020-10-01 00:00:00
train start
train_end
ranking metrics
2020-01-01 00:00:00 2020-03-20 00:00:00
train start
train_end
ranking metrics
2020-03-20 00:00:00 2020-08-01 00:00:00
train start
train_end
ranking metrics
2020-08-02 00:00:00 2020-10-01 00:00:00
train start
train_end
ranking metrics
2020-01-01 00:00:00 2020-03-20 00:00:00
train start
train_end
ranking metrics start
ranking metrics end
2020-03-20 00:00:00 2020-08-01 00:00:00
train start
train_end
ranking metrics start
ranking metrics end
2020-08-02 00:00:00 2020-10-01 00:00:00
train start
train_end
ranking metrics start
ranking metrics end
SVD
2020-01-01 00:00:00 2020-03-20 00:00:00
2020-03-20 0

In [257]:
df

,user_name,rating,unique_rest,date,city,dist,Бранч,Завтрак,Напитки,Обед,...,Электронные платежи,Азербайджанская,Словенская,Чилийская,cur_count,covid,cur_cmean_pos,cur_cmean_neu,cur_cmean_neg,user_context
0,530sofiaaa_0B701BA503208A2FB5D3AF2844D4FB06,4,250_Stockholm,2021-03-09,2,2,0,0,0,1,...,0,0,0,0,3,2,2,2,2,"[2, 2, 3, 2, 2]"
1,JohanH1529_1C18195EAB577B0E5C0B075158B3DF16,3,626_Stockholm,2021-03-09,2,0,0,0,0,1,...,0,0,0,0,11,2,3,1,2,"[3, 2, 11, 2, 1]"
2,548michaelc_AD13728165EA8E95F37CBDCF265266F5,4,290_Stockholm,2021-03-08,2,1,0,0,0,0,...,0,0,0,0,10,2,2,1,2,"[2, 2, 10, 2, 1]"
3,helen692019_9F1DA5E86A2E77191F1F825015E4EB73,5,223_Stockholm,2021-03-08,2,3,0,0,0,0,...,0,0,0,0,2,2,4,2,2,"[4, 2, 2, 2, 2]"
4,Soffitopp2_8F17C665CE80E772B0B7E11E1BC9245E,5,239_Stockholm,2021-03-07,2,1,0,0,0,1,...,0,0,0,0,1,2,4,3,0,"[4, 0, 1, 2, 3]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14789,GregorioIndelicato_1ED470D87F987CA25ED3AF34CCD...,4,404_Stockholm,2019-01-02,2,2,1,0,1,1,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0]"
14790,120fredrikl_C2E4159838661BC41F7C465CBB655535,3,547_Stockholm,2019-01-02,2,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0]"
14791,Sven_48_5D1CDAC99FBFB18D897E1A04FD114434,5,467_Stockholm,2019-01-02,2,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0]"
14792,PAMM67_E2052DC25E7B434EEE83ADC728AC7E71,3,883_Stockholm,2019-01-02,2,3,1,0,0,1,...,0,0,0,0,0,0,3,3,0,"[3, 0, 0, 0, 3]"


# Добавление новых столбцов

In [9]:
# расстояние до центра в перцентилях
def dist2category(city, dist, city_dist_dict):
    temp = []
    for c,d in zip(city, dist):
        if d<=city_dist_dict[c]['25%']:
            temp.append(0)
        elif city_dist_dict[c]['25%']<d<=city_dist_dict[c]['50%']:
            temp.append(1)
        elif city_dist_dict[c]['50%']<d<=city_dist_dict[c]['75%']:
            temp.append(2)
        else:
            temp.append(3)
    return temp

In [10]:
def cur_count2cur_count_category(x):
    if x==0:
        return 0
    if x==1:
        return 1
    if x==2:
        return 2
    if x==3:
        return 3
    if x==4:
        return 4
    if x==5:
        return 5
    if x==6:
        return 6
    if x==7:
        return 7
    if x==8:
        return 8
    if x==9:
        return 9
    if 10<=x<20:
        return 10
    if x>=20:
        return 11

In [11]:
def cur_sent2cur_sent_category(x, col, tdict):
    if x==0:
        return 0
    elif 0 < x <= tdict[col]['25%']:
        return 1
    elif tdict[col]['25%'] < x <= tdict[col]['50%']:
        return 2
    elif tdict[col]['50%'] < x <= tdict[col]['75%']:
        return 3
    else:
        return 4

In [12]:
#даты в зависимости от ковид
bounds = {'before_covid':[datetime.datetime(2020,1,1), datetime.datetime(2020,3,20)],
          'during_covid':[datetime.datetime(2020,3,20), datetime.datetime(2020,8,1)], 
          'after_covid':[datetime.datetime(2020,8,2), datetime.datetime(2020,10,1)]}
def func(date):
    if date<=bounds['before_covid'][1]:
        return 0
    if bounds['during_covid'][0]<=date<=bounds['during_covid'][1]:
        return 1
    if bounds['after_covid'][0]<=date:
        return 2

# Модели и метрики

In [13]:
import copy
def mapak(predicted, true, k):
    apaks = []
    predicted_ = copy.deepcopy(predicted)
    for u in predicted_.keys():
        if u not in true or len(predicted_[u]) == 0:
            apaks.append(0.0)
            continue
        if len(predicted_[u]) > k:
            predicted_[u] = predicted_[u][:k]
        count = 0.0
        score = 0.0
        for i, p in enumerate(predicted_[u]):
            if p in true[u] and p not in predicted_[u][:i]:
                count += 1.0
                score += count/(i+1.0)
        temp = score/min(len(predicted_[u]), k)
        apaks.append(temp)
    return np.mean(apaks)

In [14]:
def ratata(x):
    if 1<=x<1.5:
        return 0
    if 1.5<=x<2:
        return 1
    if 2<=x<2.5:
        return 2
    if 2.5<=x<3:
        return 3
    if 3<=x<3.5:
        return 4
    if 3.5<=x<4:
        return 5
    if 4<=x<4.5:
        return 6
    if 4.5<=x<=5:
        return 7
    
def func_desc(x, desc):
    if x==0:
        return 0
    elif 0 < x <= desc['25%']:
        return 1
    elif desc['25%'] < x <= desc['50%']:
        return 2
    elif desc['50%'] < x <= desc['75%']:
        return 3
    else:
        return 4

# С КОНТЕКСТОМ

In [15]:
class SVD_CONTEXT_USER():
    def __init__(self, mat, context_mat, k, iterations, gamma_1, gamma_2, gamma_3, lambda_6, lambda_7, lambda_8, cats, ncats):
        self.R = mat.to_numpy()
        self.R_ = mat.T.to_numpy()
        self.context_mat = context_mat
        self.num_users, self.num_items = self.R.shape
        self.k = k
        self.iterations = iterations

        self.users = mat.index.tolist()
        self.items = mat.columns.tolist()

        self.gamma_1 = gamma_1
        self.gamma_2 = gamma_2
        self.gamma_3 = gamma_3
        self.lambda_6 = lambda_6
        self.lambda_7 = lambda_7
        self.lambda_8 = lambda_8

        self.P = np.random.normal(scale=1. / self.k, size=(self.num_users, self.k))
        self.Q = np.random.normal(scale=1. / self.k, size=(self.num_items, self.k))
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])
        self.samples = self.get_samples()
        self.cats = cats
        self.contextNum = ncats
        self.btcj = self.get_btcj()

    def get_btcj(self):
        btcj = list()
        for i in range(self.num_users):
            contextDict = dict()
            for j in range(len(self.contextNum)):
                contextDict[j] = [0.0 for z in range(self.contextNum[j])]
            btcj.append(contextDict)
        return btcj

    def get_samples(self):
        samples = [
            (i, j, self.R[i, j], self.context_mat[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]
        return samples

    def train(self):
        print('train start')
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))

        return training_process

    def mse(self):
        xs, ys = self.R.nonzero()
        error = 0
        for x, y in zip(xs, ys):
            ctx = self.context_mat[x, y]
            error += pow(self.R[x, y] - self.get_rating(x, y, ctx), 2)
        return np.sqrt(error / len(xs))

    def sgd(self):
        for i, j, r, context in self.samples:
            prediction = self.get_rating(i, j, context)
            e = (r - prediction)

            self.b_u[i] += self.gamma_1 * (e - self.lambda_6 * self.b_u[i])
            self.b_i[j] += self.gamma_1 * (e - self.lambda_6 * self.b_i[j])

            P_i = self.P[i, :][:]
            self.P[i, :] += self.gamma_2 * (e * self.Q[j, :] - self.lambda_7 * self.P[i, :])
            self.Q[j, :] += self.gamma_2 * (e * P_i - self.lambda_7 * self.Q[j, :])

            for key, values in self.btcj[i].items():
                value = values[context[key]]
                value += self.gamma_3 * (e - self.lambda_8 * value)
                self.btcj[i][key][context[key]] = value


    def get_rating(self, i, j, context):
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
#         prediction = self.b + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)        
        for key, value in self.btcj[i].items():
            if context[key] >= 0:
                prediction += value[int(context[key])]
        return prediction

    def predict(self, user, item, context):
        prediction = self.b
        if user in self.users:
            i = self.users.index(user)
            prediction+=self.b_u[i]
            for key, value in self.btcj[i].items():
                if context[key] >= 0:
                    prediction += value[int(context[key])]
        if item in self.items:
            j = self.items.index(item)
            prediction+=self.b_i[j]
        if user in self.users and item in self.items:
            prediction+=self.P[i, :].dot(self.Q[j, :].T)
        if prediction > 5:
            return 5
        elif prediction < 1:
            return 1
        else:
            return prediction
    
    def predict_all(self, user, nrests, contexts):
        i = self.users.index(user)
        pred1 = np.full(nrests, self.b)
        pred2 = np.full(nrests, self.b_u[i])
        pred3 = self.P[i, :].dot(self.Q.T)
        pred4 = self.b_i
        pred5 = []
        for ii, x in enumerate(contexts):
            if ii > nrests-1:
                break
            pred5.append((sum([v[int(x[k])] for k,v in self.btcj[i].items()])))
        return pred1+pred2+pred3+pred5

In [16]:
class SVD_CONTEXT_ITEM():
    def __init__(self, mat, context_mat, k, iterations, 
                 gamma_1, gamma_2, gamma_3,
                 lambda_6, lambda_7, lambda_8,
                 cats, ncats):
        self.R = mat.to_numpy()
        self.R_ = mat.T.to_numpy()
        self.context_mat = context_mat
        self.num_users, self.num_items = self.R.shape
        self.k = k
        self.iterations = iterations

        self.users = mat.index.tolist()
        self.items = mat.columns.tolist()

        self.gamma_1 = gamma_1
        self.gamma_2 = gamma_2
        self.gamma_3 = gamma_3
        self.lambda_6 = lambda_6
        self.lambda_7 = lambda_7
        self.lambda_8 = lambda_8

        self.P = np.random.normal(scale=1. / self.k, size=(self.num_users, self.k))
        self.Q = np.random.normal(scale=1. / self.k, size=(self.num_items, self.k))
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])
        self.samples = self.get_samples()
        self.cats = cats
        self.ncats = ncats
        self.contextNum = ncats
        self.btcj = self.get_btcj()

    def get_btcj(self):
        btcj = list()
        for i in range(self.num_items):
            contextDict = dict()
            for j in range(len(self.contextNum)):
                contextDict[j] = [0.0 for z in range(self.contextNum[j])]
            btcj.append(contextDict)
        return btcj

    def get_samples(self):
        samples = [
            (i, j, self.R[i, j], self.context_mat[j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]
        return samples

    def train(self):
        print('train start')
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))

        return training_process

    def mse(self):
        xs, ys = self.R.nonzero()
        error = 0
        for x, y in zip(xs, ys):
            ctx = self.context_mat[y]
            error += pow(self.R[x, y] - self.get_rating(x, y, ctx), 2)
        return np.sqrt(error / len(xs))

    def sgd(self):
        for i, j, r, context in self.samples:
            prediction = self.get_rating(i, j, context)
            e = (r - prediction)

            self.b_u[i] += self.gamma_1 * (e - self.lambda_6 * self.b_u[i])
            self.b_i[j] += self.gamma_1 * (e - self.lambda_6 * self.b_i[j])

            P_i = self.P[i, :][:]
            self.P[i, :] += self.gamma_2 * (e * self.Q[j, :] - self.lambda_7 * self.P[i, :])
            self.Q[j, :] += self.gamma_2 * (e * P_i - self.lambda_7 * self.Q[j, :])

            for key, values in self.btcj[j].items():
                value = values[context[key]]
                value += self.gamma_3 * (e - self.lambda_8 * value)
                self.btcj[j][key][context[key]] = value


    def get_rating(self, i, j, context):
#         prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        prediction = self.b + self.b_u[i] + self.P[i, :].dot(self.Q[j, :].T)
        for key, value in self.btcj[j].items():
            if context[key] >= 0:
                prediction += value[int(context[key])]
            else:
                prediction += self.b_i[j]
        return prediction

    def predict(self, user, item, context):
        prediction = self.b
        if user in self.users:
            i = self.users.index(user)
            prediction+=self.b_u[i]
        if item in self.items:
            j = self.items.index(item)
            for key, value in self.btcj[j].items():
                if context[key] >= 0:
                    prediction += value[int(context[key])]
                else:
                    prediction+=self.b_i[j]
        if user in self.users and item in self.items:
            prediction+=self.P[i, :].dot(self.Q[j, :].T)
        if prediction > 5:
            return 5
        elif prediction < 1:
            return 1
        else:
            return prediction
        
        
    def predict_all(self, user, nrests):
        i = self.users.index(user)
        pred1 = np.full(nrests, self.b)
        pred2 = np.full(nrests, self.b_u[i])
        pred3 = self.P[i, :].dot(self.Q.T)
        pred4 = self.b_i
        pred5 = []
        for ii, x in enumerate(self.context_mat):
            if ii > nrests-1:
                break
            pred5.append((sum([v[int(x[k])] for k,v in self.btcj[ii].items()])))
        return pred1+pred2+pred3+pred5

In [17]:
class CAMF_CC():
    def __init__(self, mat, context_mat, k, iterations, 
                 gamma_1, gamma_2, gamma_3, 
                 lambda_6, lambda_7, lambda_8, 
                 cats, ncats, categoryNum, gr_index):
        self.R = mat.to_numpy()
        self.R_ = mat.T.to_numpy()
        self.context_mat = context_mat
        self.num_users, self.num_items = self.R.shape
        self.k = k
        self.iterations = iterations

        self.users = mat.index.tolist()
        self.items = mat.columns.tolist()

        self.gamma_1 = gamma_1
        self.gamma_2 = gamma_2
        self.gamma_3 = gamma_3
        self.lambda_6 = lambda_6
        self.lambda_7 = lambda_7
        self.lambda_8 = lambda_8

        self.P = np.random.normal(scale=1. / self.k, size=(self.num_users, self.k))
        self.Q = np.random.normal(scale=1. / self.k, size=(self.num_items, self.k))
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])
        self.samples = self.get_samples()
        self.cats = cats
        self.ncats = ncats
        self.contextNum = ncats
        self.categoryNum = categoryNum
        self.btcj = self.get_btcj()
        self.gr_index = gr_index

    def get_btcj(self):
        btcj = list()
        for i in range(self.categoryNum):
            contextDict = dict()
            for j in range(len(self.contextNum)):
                contextDict[j] = [0.0 for z in range(self.contextNum[j])]
            btcj.append(contextDict)
        return btcj

    def get_samples(self):
        samples = [
            (i, j, self.R[i, j], self.context_mat[j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]
        return samples

    def train(self):
        print('train start')
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))

        return training_process

    def mse(self):
        xs, ys = self.R.nonzero()
        error = 0
        for x, y in zip(xs, ys):
            ctx = self.context_mat[y]
            error += pow(self.R[x, y] - self.get_rating(x, y, ctx), 2)
        return np.sqrt(error / len(xs))

    def sgd(self):
        for i, j, r, context in self.samples:
            prediction = self.get_rating(i, j, context)
            e = (r - prediction)

            self.b_u[i] += self.gamma_1 * (e - self.lambda_6 * self.b_u[i])
            self.b_i[j] += self.gamma_1 * (e - self.lambda_6 * self.b_i[j])

            P_i = self.P[i, :][:]
            self.P[i, :] += self.gamma_2 * (e * self.Q[j, :] - self.lambda_7 * self.P[i, :])
            self.Q[j, :] += self.gamma_2 * (e * P_i - self.lambda_7 * self.Q[j, :])
            
            cid = self.context_mat[j, self.gr_index]
            for key, values in self.btcj[cid].items():
                value = values[context[key]]
                value += self.gamma_3 * (e - self.lambda_8 * value)
                self.btcj[cid][key][context[key]] = value


    def get_rating(self, i, j, context):
        prediction = self.b + self.b_u[i] + self.P[i, :].dot(self.Q[j, :].T)
        cid = context[self.gr_index]
        for key, value in self.btcj[cid].items():
            if context[key] >= 0:
                prediction += value[int(context[key])]
            else:
                prediction += self.b_i[j]
        return prediction

    def predict(self, user, item, context):
        prediction = self.b
        cid = context[self.gr_index]
        for key, value in self.btcj[cid].items():
            if context[key] >= 0:
                prediction += value[int(context[key])]
        if item in self.items:
            j = self.items.index(item)
#             prediction+=self.b_i[j]
        if user in self.users:
            i = self.users.index(user)
            prediction+=self.b_u[i]
        if user in self.users and item in self.items:
            prediction+=self.P[i, :].dot(self.Q[j, :].T)
        if prediction > 5:
            return 5
        elif prediction < 1:
            return 1
        else:
            return prediction
    
    
    def predict_all(self, user, nrests):
        i = self.users.index(user)
        pred1 = np.full(nrests, self.b)
        pred2 = np.full(nrests, self.b_u[i])
        pred3 = self.P[i, :].dot(self.Q.T)
        pred4 = self.b_i
        pred5 = []
        for ii, x in enumerate(self.context_mat):
            cid = x[self.gr_index]
            if ii > nrests-1:
                break
            pred5.append((sum([v[int(x[k])] for k,v in self.btcj[cid].items()])))
        return pred1+pred2+pred3+pred5

In [18]:
def create_train_test(df,date_start,date_end):
    df_train = df[(df.date.between(date_start-dateutil.relativedelta.relativedelta(months=6), 
                                   date_start-dateutil.relativedelta.relativedelta(days=1)))]
    df_test = df[(df.date.between(date_start, date_end))]
    #пользователи и рестораны только те которые в трейне
    df_test = df_test[(df_test.user_name.isin(df_train.user_name.unique().tolist()))
                       &(df_test.unique_rest.isin(df_train.unique_rest.unique().tolist()))]
    return df_train,df_test

In [ ]:
def svd_user(df,bounds,city):
    ndcg = NDCG(10)
    svd_user_pred = pd.DataFrame()
    for i,k in enumerate(bounds.keys()):
        print(bounds[k][0], bounds[k][1])
        met = pd.DataFrame()
        df_train,df_test=create_train_test(df,bounds[k][0], bounds[k][1])
        mat_train = df_train.pivot(index='user_name', columns='unique_rest', values='rating').fillna(0)
        mat_train = mat_train.sort_values('user_name')
        mat_train = mat_train.reindex(sorted(mat_train.columns), axis=1)
        user_context_mat = df_train.pivot(index='user_name', columns='unique_rest', values='user_context').fillna(0)
        user_context_mat = user_context_mat.sort_values('user_name')
        user_context_mat = user_context_mat.reindex(sorted(mat_train.columns), axis=1)
        user_context_mat.fillna(0, inplace=True)
        user_context_mat_ = user_context_mat.to_numpy()
        user_context_mat_test = df_test.pivot(index='user_name', columns='unique_rest', values='user_context').fillna(0)
        user_context_mat_test = user_context_mat_test.sort_values('user_name')
        user_context_mat_test.fillna(0, inplace=True)
        user_context_mat_test_ = user_context_mat_test.to_numpy()
        mdl = SVD_CONTEXT_USER(mat = mat_train,context_mat = user_context_mat_,k = 100,iterations = 20,
                      gamma_1 = 0.005,gamma_2 = 0.005, gamma_3 = 0.005, lambda_6 = 0.02, lambda_7 = 0.02,
                      lambda_8 = 0.02,cats=cats_user, ncats=ncats_user)
        mdl.train()
        print('train_end')
        rate_ = []
        for x, y in zip(df_test['user_name'].tolist(), df_test['unique_rest'].tolist()):
             rate_.append(mdl.predict(x, y, user_context_mat_test_[user_context_mat_test.index.tolist().index(x),
                                                              user_context_mat_test.columns.tolist().index(y)]))
        rmse = mean_squared_error(y_pred=rate_, y_true=df_test.rating, squared=False)
        mae = mean_absolute_error(y_pred=rate_, y_true=df_test.rating)
        print('ranking metrics')
        
        df_test_ = df_train.pivot(index='user_name', columns='unique_rest', values='rating').fillna(0)
        df_test_ = df_test_[df_test_.index.isin(df_test.user_name.unique().tolist())]
        df_test_.update(df_test.pivot(index='user_name', columns='unique_rest', values='rating').fillna(0))
    
        df_train_ = df_train.pivot(index='user_name', columns='unique_rest', values='rating').fillna(0)
        df_train_ = df_train_[df_train_.index.isin(df_train.user_name.unique().tolist())]
        df_test_dict = df_test_.T.to_dict('list')
    
        user_context_mat.update(user_context_mat_test)
        us = {}
        for u in df_test_.index.tolist():
            temp = df_test.sort_values('date').groupby('user_name')['user_context'].agg('first')
            user_contexts = [x if x!=0 else temp[u] for x in user_context_mat.loc[u].tolist()]
            values = mdl.predict_all(u, len(df_train_.columns), user_contexts)
            iis = [i for i,x in enumerate(df_train_.loc[u].to_list()) if x!=0]
            np.put(values, iis, [0 for x in iis])
            us[u] = values
        ndcg_metric = {}
        for kk in us.keys():
            ndcg_metric[kk] = ndcg.compute(df_test_dict[kk], np.array(us[kk]).argsort()[::-1])
        ndcg_ = np.mean(list(ndcg_metric.values()))
        mapak10 = mapak(us, df_test_dict, 10)
        met = met.append({
            'alg':'SVD_USER',
            'period': (str(bounds[k][0])[:7]+' - '+str(bounds[k][1])[:7]),
            "city":city,
            'rmse': rmse,
            'mae': mae,
            'ndcg10': ndcg_,
            'mapak10': mapak10
        }, ignore_index=True)
        svd_user_pred = svd_user_pred.append(met, ignore_index=True)
    return svd_user_pred     

In [ ]:
def svd_context(df,bounds,сast_rest,city):
    ndcg = NDCG(10)
    svd_context_pred = pd.DataFrame()
    for i,k in enumerate(bounds.keys()):
        print(bounds[k][0], bounds[k][1])
        met = pd.DataFrame()
        df_train,df_test=create_train_test(df,bounds[k][0], bounds[k][1])
        #средний рейтинг
        temp = df[df.date<bounds[k][0]].groupby('unique_rest')['rating'].mean().reset_index().rename(columns={'rating':'average_rating'})
        temp['average_rating'] = temp.average_rating.apply(lambda x: ratata(x))
        temp_avg_rating2cat_dict = {x:i for i,x in enumerate(temp['average_rating'].unique().tolist())}
        temp['average_rating'] = temp['average_rating'].apply(lambda x: temp_avg_rating2cat_dict[x])
        #количество оценок
        temp1 = df[df.date<bounds[k][0]].groupby('unique_rest')['rating'].count().reset_index().rename(columns={'rating':'num_all_rev'})
        temp1['num_all_rev'] = temp1.num_all_rev.apply(lambda x: func_desc(x, temp1['num_all_rev'].describe()))
        temp1_avg_rating2cat_dict = {x:i for i,x in enumerate(temp1['num_all_rev'].unique().tolist())}
        temp1['num_all_rev'] = temp1['num_all_rev'].apply(lambda x: temp1_avg_rating2cat_dict[x])
        
        df_train = df_train.merge(temp, on='unique_rest')
        df_test = df_test.merge(temp, on='unique_rest')
        df_train = df_train.merge(temp1, on='unique_rest')
        df_test = df_test.merge(temp1, on='unique_rest')
        mat_train = df_train.pivot(index='user_name', columns='unique_rest', values='rating').fillna(0)
        mat_train = mat_train.sort_values('user_name')
        mat_train = mat_train.reindex(sorted(mat_train.columns), axis=1)
        rests = df_train.drop_duplicates('unique_rest').reset_index()
#     rests = df.drop_duplicates('unique_rest').reset_index().merge(temp, on='unique_rest')
        rests = rests[['unique_rest'] + cats_rest]
        ncats = [rests[x].nunique() for x in cats_rest]
        name_index_rest = {i:x for i,x in enumerate(rests.unique_rest.unique())}
        nrests = len(name_index_rest)
        mat_mat = rests.set_index('unique_rest')[cats_rest].sort_index()
        mat_mat.fillna(0, inplace=True)
        mat_mat_ = mat_mat.to_numpy()
        mdl = SVD_CONTEXT_ITEM(mat = mat_train,context_mat = mat_mat_,k = 100,iterations = 20,gamma_1 = 0.005,
                  gamma_2 = 0.005,gamma_3 = 0.005,lambda_6 = 0.02,lambda_7 = 0.02,lambda_8 = 0.02,cats=cats_rest,ncats=ncats)
        mdl.train()
        print('train_end')
        pred = []
        for x, y in zip(df_test['user_name'].tolist(), df_test['unique_rest'].tolist()):
            pred.append(mdl.predict(x, y, mat_mat_[mat_mat.index.tolist().index(y)]))
        rmse = mean_squared_error(y_pred=pred, y_true=df_test.rating, squared=False)
        mae = mean_absolute_error(y_pred=pred, y_true=df_test.rating)
        df_test_ = df_train.pivot(index='user_name', columns='unique_rest', values='rating').fillna(0)
        df_test_ = df_test_[df_test_.index.isin(df_test.user_name.unique().tolist())]
        df_test_.update(df_test.pivot(index='user_name', columns='unique_rest', values='rating').fillna(0))
        df_train_ = df_train.pivot(index='user_name', columns='unique_rest', values='rating').fillna(0)
        df_train_ = df_train_[df_train_.index.isin(df_train.user_name.unique().tolist())]
        df_test_dict = df_test_.T.to_dict('list')
        print('ranking metrics')
        us = {}
        for u in df_test_.index.tolist():
            values = mdl.predict_all(u, len(df_train_.columns))
            iis = [i for i,x in enumerate(df_train_.loc[u].to_list()) if x!=0]
            np.put(values, iis, [0 for x in iis])
            us[u] = values
        ndcg_metric = {}
        for kk in us.keys():
            ndcg_metric[kk] = ndcg.compute(df_test_dict[kk], np.array(us[kk]).argsort()[::-1])
        ndcg_ = np.mean(list(ndcg_metric.values()))
        mapak10 = mapak(us, df_test_dict, 10)   
        met = met.append({
            'alg':'SVD_CONTEXT',
            'period': (str(bounds[k][0])[:7]+' - '+str(bounds[k][1])[:7]),
            "city":city,
            'rmse': rmse,
            'mae': mae,
            'ndcg10': ndcg_,
            'mapak10': mapak10
             }, ignore_index=True)
        svd_context_pred = svd_context_pred.append(met, ignore_index=True)
    return svd_context_pred 

In [285]:
group_index = cats_rest.index('average_rating')
met = pd.DataFrame()
df_train,df_test=create_train_test(df,bounds[k][0], bounds[k][1])
temp = df[df.date<bounds[k][0]].groupby('unique_rest')['rating'].mean().reset_index().rename(columns={'rating':'average_rating'})
temp['average_rating'] = temp.average_rating.apply(lambda x: ratata(x))
temp_avg_rating2cat_dict = {x:i for i,x in enumerate(temp['average_rating'].unique().tolist())}
temp['average_rating'] = temp['average_rating'].apply(lambda x: temp_avg_rating2cat_dict[x])
temp1 = df[df.date<bounds[k][0]].groupby('unique_rest')['rating'].count().reset_index().rename(columns={'rating':'num_all_rev'})
temp1['num_all_rev'] = temp1.num_all_rev.apply(lambda x: func_desc(x, temp1['num_all_rev'].describe()))
temp1_avg_rating2cat_dict = {x:i for i,x in enumerate(temp1['num_all_rev'].unique().tolist())}
temp1['num_all_rev'] = temp1['num_all_rev'].apply(lambda x: temp1_avg_rating2cat_dict[x])
df_train = df_train.merge(temp, on='unique_rest')
df_test = df_test.merge(temp, on='unique_rest')
df_train = df_train.merge(temp1, on='unique_rest')
df_test = df_test.merge(temp1, on='unique_rest')
mat_train = df_train.pivot(index='user_name', columns='unique_rest', values='rating').fillna(0)
mat_train = mat_train.sort_values('user_name')
mat_train = mat_train.reindex(sorted(mat_train.columns), axis=1)
rests = df_train.drop_duplicates('unique_rest').reset_index()
rests = rests[['unique_rest'] + cats_rest]
ncats = [rests[x].nunique() for x in cats_rest]
name_index_rest = {i:x for i,x in enumerate(rests.unique_rest.unique())}
nrests = len(name_index_rest)
mat_mat = rests.set_index('unique_rest')[cats_rest].sort_index()
mat_mat.fillna(0, inplace=True)
mat_mat_ = mat_mat.to_numpy()

In [288]:
df_train[cats_rest[group_index]]

0       1
1       3
2       3
3       3
4       3
       ..
1786    3
1787    2
1788    2
1789    0
1790    3
Name: average_rating, Length: 1791, dtype: int64

In [246]:
def cc(df,bounds,сast_rest,city):
    svd_cс_pred_2 = pd.DataFrame()
    ndcg = NDCG(10)
    group_index = cats_rest.index('average_rating')
    for i,k in enumerate(bounds.keys()):
        print(bounds[k][0], bounds[k][1])
        met = pd.DataFrame()
        df_train,df_test=create_train_test(df,bounds[k][0], bounds[k][1])
        temp = df[df.date<bounds[k][0]].groupby('unique_rest')['rating'].mean().reset_index().rename(columns={'rating':'average_rating'})
        temp['average_rating'] = temp.average_rating.apply(lambda x: ratata(x))
        temp_avg_rating2cat_dict = {x:i for i,x in enumerate(temp['average_rating'].unique().tolist())}
        temp['average_rating'] = temp['average_rating'].apply(lambda x: temp_avg_rating2cat_dict[x])
        temp1 = df[df.date<bounds[k][0]].groupby('unique_rest')['rating'].count().reset_index().rename(columns={'rating':'num_all_rev'})
        temp1['num_all_rev'] = temp1.num_all_rev.apply(lambda x: func_desc(x, temp1['num_all_rev'].describe()))
        temp1_avg_rating2cat_dict = {x:i for i,x in enumerate(temp1['num_all_rev'].unique().tolist())}
        temp1['num_all_rev'] = temp1['num_all_rev'].apply(lambda x: temp1_avg_rating2cat_dict[x])
    
        df_train = df_train.merge(temp, on='unique_rest')
        df_test = df_test.merge(temp, on='unique_rest')
        df_train = df_train.merge(temp1, on='unique_rest')
        df_test = df_test.merge(temp1, on='unique_rest')
        mat_train = df_train.pivot(index='user_name', columns='unique_rest', values='rating').fillna(0)
        mat_train = mat_train.sort_values('user_name')
        mat_train = mat_train.reindex(sorted(mat_train.columns), axis=1)

        rests = df_train.drop_duplicates('unique_rest').reset_index()
        rests = rests[['unique_rest'] + cats_rest]
        ncats = [rests[x].nunique() for x in cats_rest]
        name_index_rest = {i:x for i,x in enumerate(rests.unique_rest.unique())}
        nrests = len(name_index_rest)

        mat_mat = rests.set_index('unique_rest')[cats_rest].sort_index()
        mat_mat.fillna(0, inplace=True)
        mat_mat_ = mat_mat.to_numpy()
    
        mdl = CAMF_CC(mat = mat_train,
                  context_mat = mat_mat_,
                  k = 100,
                  iterations = 20,
                  gamma_1 = 0.005,
                  gamma_2 = 0.005,
                  gamma_3 = 0.005,
                  lambda_6 = 0.02,
                  lambda_7 = 0.02,
                  lambda_8 = 0.02,
                  cats=cats_rest,
                  ncats=ncats,
                  categoryNum=df_train[cats_rest[group_index]].nunique(),
                  gr_index=group_index)
        mdl.train()
        print('train_end')
        pred = []
        for x, y in zip(df_test['user_name'].tolist(), df_test['unique_rest'].tolist()):
            pred.append(mdl.predict(x, y, mat_mat_[mat_mat.index.tolist().index(y)]))
        rmse = mean_squared_error(y_pred=pred, y_true=df_test.rating, squared=False)
        mae = mean_absolute_error(y_pred=pred, y_true=df_test.rating)
    
        df_test_ = df_train.pivot(index='user_name', columns='unique_rest', values='rating').fillna(0)
        df_test_ = df_test_[df_test_.index.isin(df_test.user_name.unique().tolist())]
        df_test_.update(df_test.pivot(index='user_name', columns='unique_rest', values='rating').fillna(0))
    
        df_train_ = df_train.pivot(index='user_name', columns='unique_rest', values='rating').fillna(0)
        df_train_ = df_train_[df_train_.index.isin(df_train.user_name.unique().tolist())]
        df_test_dict = df_test_.T.to_dict('list')
        print('ranking metrics start')
        us = {}
        for u in df_test_.index.tolist():
            values = mdl.predict_all(u, len(df_train_.columns))
            iis = [i for i,x in enumerate(df_train_.loc[u].to_list()) if x!=0]
            np.put(values, iis, [0 for x in iis])
            us[u] = values
        ndcg_metric = {}
        for kk in us.keys():
            ndcg_metric[kk] = ndcg.compute(df_test_dict[kk], np.array(us[kk]).argsort()[::-1])
        ndcg_ = np.mean(list(ndcg_metric.values()))
        mapak10 = mapak(us, df_test_dict, 10)
        print('ranking metrics end')
        met = met.append({
            'alg':'SVD_CONTEXT_CC_AR',
            'period': (str(bounds[k][0])[:7]+' - '+str(bounds[k][1])[:7]),
            "city":city,
            'rmse': rmse,
            'mae': mae,
            'ndcg10': ndcg_,
            'mapak10': mapak10
            }, ignore_index=True)
        svd_cс_pred_2 = svd_cс_pred_2.append(met, ignore_index=True)
    return svd_cс_pred_2

In [248]:
svd_cс_pred_2=cc(df,bounds,cats_rest,city)

2020-01-01 00:00:00 2020-03-20 00:00:00
train start
train_end
ranking metrics start
ranking metrics end
2020-03-20 00:00:00 2020-08-01 00:00:00
train start
train_end
ranking metrics start
ranking metrics end
2020-08-02 00:00:00 2020-10-01 00:00:00
train start
train_end
ranking metrics start
ranking metrics end


In [108]:
svd_user_pred=svd_user(df,bounds)
svd_context_pred=svd_context(df,bounds,cats_rest)
svd_cс_pred_2=(df,bounds,cats_rest)

2020-01-01 00:00:00 2020-03-20 00:00:00
train start
train_end
ranking metrics
2020-03-20 00:00:00 2020-08-01 00:00:00
train start
train_end
ranking metrics
2020-08-02 00:00:00 2020-10-01 00:00:00
train start
train_end
ranking metrics


In [132]:
svd_context_pred=svd_context(df,bounds,cats_rest)

2020-01-01 00:00:00 2020-03-20 00:00:00
train start
train_end
ranking metrics
2020-03-20 00:00:00 2020-08-01 00:00:00
train start
train_end
ranking metrics
2020-08-02 00:00:00 2020-10-01 00:00:00
train start
train_end
ranking metrics


# Стандартные методы

In [227]:
def usual_methods(df,bounds,сast_rest,city):
    results_surprise = pd.DataFrame()
    algos = {
        'SVD': SVD(),
        'NMF': NMF(),
        'SlopeOne' : SlopeOne(),
    }
    ndcg = NDCG(10)
    all_pred = pd.DataFrame()
    for alg in algos:
        print(alg)
        for i,k in enumerate(bounds.keys()):
            print(bounds[k][0], bounds[k][1])
            met = pd.DataFrame()
            df_train,df_test=create_train_test(df,bounds[k][0], bounds[k][1])
            df_train=df_train[['user_name', 'unique_rest', 'rating']]
            df_test=df_test[['user_name', 'unique_rest', 'rating']]
            reader = Reader(rating_scale=(1, 5))
            data_do_train = Dataset.load_from_df(df_train, reader)
            trainset = data_do_train.build_full_trainset()
            algo = algos[alg]
            algo.fit(trainset)  
            df_test_ = df_train.pivot(index='user_name', columns='unique_rest', values='rating').fillna(0)
            df_test_ = df_test_[df_test_.index.isin(df_test.user_name.unique().tolist())]
            df_test_.update(df_test.pivot(index='user_name', columns='unique_rest', values='rating').fillna(0))
            df_train_ = df_train.pivot(index='user_name', columns='unique_rest', values='rating').fillna(0)
            df_train_ = df_train_[df_train_.index.isin(df_train.user_name.unique().tolist())]
            df_test_dict = df_test_.T.to_dict('list')
            us = {}
            for u in df_test_.index.tolist():
                values = [algo.predict(u, i).est if df_train_.loc[u, i]==0 else 0 for i in df_test_.columns.tolist()]
                us[u] = values
            ndcg_metric = {}
            for kk in us.keys():
                ndcg_metric[kk] = ndcg.compute(df_test_dict[kk], np.array(us[kk]).argsort()[::-1])
            ndcg_ = np.mean(list(ndcg_metric.values()))     
            test_ui_dict = df_test.groupby('user_name')['unique_rest'].apply(list).to_dict()
            test_ur_dict = df_test.groupby('user_name')['rating'].apply(list).to_dict()
            pred_ur_dict_cut = {}
            pred_ur_dict_cut_ = {}
            for u in test_ui_dict.keys():
                pred_ur_dict_cut[u] = [algo.predict(uid=u, iid=i).est for i in test_ui_dict[u]]# if len(test_ui_dict[u])>1]
            for u in test_ui_dict.keys():
                if len(test_ui_dict[u])>1:
                    pred_ur_dict_cut_[u] = [algo.predict(uid=u, iid=i).est for i in test_ui_dict[u]]
            ndcg_metric_cut = {}
            ndcg_metric_cut_ = {}
            for u in pred_ur_dict_cut.keys():
                ndcg_metric_cut[u] = ndcg.compute(test_ur_dict[u], np.array(pred_ur_dict_cut[u]).argsort()[::-1])
            for u in pred_ur_dict_cut_.keys():
                ndcg_metric_cut_[u] = ndcg.compute(test_ur_dict[u], np.array(pred_ur_dict_cut_[u]).argsort()[::-1])
            ndcg_10_cut = np.mean(list(ndcg_metric_cut.values()))
            ndcg_10_cut_ = np.mean(list(ndcg_metric_cut_.values()))
            mapak10 = mapak(us, df_test_dict, 10)     
            rate_ = []
            for x, y in zip(df_test['user_name'].tolist(), df_test['unique_rest'].tolist()):
                rate_.append([algo.predict(uid=x,iid=y).est])
            rmse = mean_squared_error(y_pred=rate_, y_true=df_test.rating, squared = False)
            mae = mean_absolute_error(y_pred=rate_, y_true=df_test.rating)
            met = met.append({
                'alg':alg,
                'period': (str(bounds[k][0])[:7]+' - '+str(bounds[k][1])[:7]),
                "city":city,
                'rmse': rmse,
                'mae': mae,
                'ndcg10': ndcg_,
                'ndcg10_cut': ndcg_10_cut,
                'ndcg10_cut_': ndcg_10_cut_,
                'mapak10': mapak10
            }, ignore_index=True)
            all_pred = all_pred.append(met, ignore_index=True)
    return all_pred    